# 자연어 처리 (NLP)
 > 자연어 처리 및 분석이란 텍스트 데이터와 같은 비정형의 자연어를 이해하고 해석하여, 이에 관한 인사이트를 도출해내는 분석 기법을 의미합니다.

## 목차
- import Base code
    - word cloud / 단어 빈도 분석
    - word_relative / 연관어 분석
    - key_sentence / 핵심 문장 추출
- 함수 시연
    - word cloud / 단어 빈도 분석
    - word_relative / 연관어 분석
    - key_sentence / 핵심 문장 추출

## import Base code

In [ ]:
! pip install textrank==0.1.2

In [11]:
import pandas as pd

from collections import Counter
from konlpy.tag import Mecab
from nltk import sent_tokenize

## word cloud / 단어 빈도 분석
> 단어 빈도 분석이란 특정 문서에서 자주 사용되는 단어를 추출해내어, 빈도 수에 따라 중요도를 분석하는 방법을 말합니다.

- 구성
    - word_cloud 실행함수

In [12]:
# word_cloud 실행함수

def word_cloud(lines):
    noun_list = Mecab().nouns(lines)
    df = pd.DataFrame([[k, v] for k, v in dict(Counter(noun_list)).items()], columns=["유형", "개수"])
    df = df.sort_values("개수", ascending=False).head(50)
    return {
        "fields": [
            {"name": "유형", "type": "TEXT"},
            {"name": "개수", "type": "INTEGER"}],
        "results": df.values.tolist()
    }

## word_relative / 연관어 분석
> 연관어 분석은 문서 내 추출된 단어들이 동시에 출현한 빈도를 계산해서, 키워드 간의 연관도에 따른 네트워크 구성을 보여주는 시각화 기법입니다.

- 구성
  - word_relative 실행함수
  - word_relative main logic

In [13]:
# word_relative 실행함수

def word_relative(lines):
    nlp_word_relative = NlpWordRelative()
    nlp_word_relative.lines2sentences(lines)
    df = nlp_word_relative.start()
    return {
        "fields": [
            {"name": "id", "type": "TEXT"},
            {"name": "link", "type": "TEXT"},
            {"name": "radius", "type": "INTEGER"},
            {"name": "color", "type": "TEXT"}],
        "results": df.values.tolist()
    }

In [14]:
# word_relative main logic

from colorutils import Color


class NlpWordRelative:
    def __init__(self):
        self.key_list = {}
        self.group_list = {}
        self.colors = [0, 60, 120, 240, 300]
        self.lines = None

    def create_radius_df(self, words_list, n):
        count = {}
        for line in words_list:
            words = list(set(line.split()))
            for i, a in enumerate(words):
                for b in words[i + 1:]:
                    if a > b:
                        count[b, a] = count.get((b, a), 0) + 1
                    else:
                        count[a, b] = count.get((a, b), 0) + 1

        count.get(("a", "b"), 0)
        df = pd.DataFrame.from_dict(count, orient='index')

        edge_list = []
        for i in range(len(df)):
            edge_list.append([df.index[i][0], df.index[i][1], df[0][i], '#bfbfbf'])

        df2 = pd.DataFrame(edge_list, columns=["id", "link", "radius", "color"])\
            .sort_values(by=['radius'], ascending=False)

        radius_df = df2.head(n)
        return radius_df

    def combine_group(self, l_idx, r_idx=None, new_group=None):
        if r_idx:
            for k, v in self.key_list.items():
                if v == r_idx:
                    self.key_list[k] = l_idx
            new_group = self.group_list.pop(r_idx)
        for k, v in new_group.items():
            if k in self.group_list[l_idx].keys():
                self.group_list[l_idx][k] = self.group_list[l_idx][k] + v
            else:
                self.group_list[l_idx][k] = v

    def key_check(self, set_id, w):
        if w[0] not in self.key_list and w[1] not in self.key_list:
            set_id += 1  # new set_id
            self.key_list[w[0]] = set_id
            self.key_list[w[1]] = set_id
            return set_id, None
        elif w[0] in self.key_list and w[1] not in self.key_list:
            self.key_list[w[1]] = self.key_list[w[0]]
            return self.key_list[w[0]], None
        elif w[1] in self.key_list and w[0] not in self.key_list:
            self.key_list[w[0]] = self.key_list[w[1]]
            return self.key_list[w[1]], None
        else:
            if self.key_list[w[0]] == self.key_list[w[1]]:
                return self.key_list[w[0]], None
            else:
                return self.key_list[w[0]], self.key_list[w[1]]

    def hsv_to_hex(self, h, s, v):
        c = Color(hsv=(h, s / 100, v / 100))
        red, green, blue = c.rgb
        ir, ig, ib = int(red), int(green), int(blue)
        return '#' + hex(ir)[2:].zfill(2) + hex(ig)[2:].zfill(2) + hex(ib)[2:].zfill(2)

    def create_vertex_list(self, nodes, hue):
        nodes_dict = dict(sorted(nodes.items(), key=lambda x: x[1], reverse=True))
        node_list = []
        s = 100
        before = 0
        for k, v in nodes_dict.items():
            if before > v:
                s -= 10
            before = v
            node_list.append([k, k, v, self.hsv_to_hex(hue, s, 100)])
        node_list_with_color = pd.DataFrame(node_list, columns=["id", "link", "radius", "color"])
        return node_list_with_color

    def manage_scale(self, nodes):
        for k, v in nodes.items():
            if v > 50:
                nodes[k] = 50
        return nodes

    def lines2sentences(self, lines):   # 문장의 끝맺음이 제대로 되지 않을때 문장 분리
        sentences = sent_tokenize(lines)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx - 1] += (' ' + sentences[idx])
                sentences[idx] = ''
        self.lines = sentences

    def read_file(self, path):  # 문장의 끝맺음이 제대로 될때 사용시 성능 향상
        text_file = open(path, "r")
        file_content = text_file.read()

        content_list = file_content.split("\n")
        text_file.close()

        self.lines = content_list

    def start(self):
        words_list = []
        for text in self.lines:
            words_list.append(text.strip())

        radius_df = self.create_radius_df(words_list, 100)

        i = 0
        for index, row in radius_df.iterrows():
            w1 = row['id']
            w2 = row['link']
            l_set_idx, r_set_idx = self.key_check(i, [w1, w2])
            group = {}

            if r_set_idx:
                self.combine_group(l_set_idx, r_set_idx, None)
                self.group_list[l_set_idx][w1] += row['radius']
                self.group_list[l_set_idx][w2] += row['radius']
            else:
                if l_set_idx not in self.group_list:
                    group[w1] = row['radius']
                    group[w2] = row['radius']
                    self.group_list[l_set_idx] = group
                else:
                    group[w1] = row['radius']
                    group[w2] = row['radius']
                    self.combine_group(l_set_idx, None, group)
            i += 1

        frames = []
        j = 0
        for k, v in self.group_list.items():
            frames.append(self.create_vertex_list(self.manage_scale(v), self.colors[j]))
            j += 1

        frames.append(radius_df)
        result = pd.concat(frames)

        return result


## key_sentence / 핵심 문장 추출
> 핵심 문장 추출을 통해 문서를 요약할 수 있는 핵심이 되는 내용을 확인할 수 있습니다.

- 구성
  - key_sentence 실행함수
  - key_sentence main logic

In [15]:
# key_sentence 실행함수

def key_sentence(lines):
    nlp_sentence = NlpSentence()
    nlp_sentence.lines2sentences(lines)
    nlp_sentence.set_summary_line_cnt(5)

    runner = nlp_sentence.run()
    return {
        "fields": [
            {"name": "핵심 문장 추출", "type": "TEXT"}],
        "results": [[res[2]] for res in runner]
    }

In [16]:
# key_sentence main logic

from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer


class NlpSentence:
    def __init__(self):
        self.komoran = Komoran()
        self.sents = None
        self.cnt = 6

    def read_file(self, path):  # 문장의 끝맺음이 제대로 될때 사용시 성능 향상
        text_file = open(path, "r")
        file_content = text_file.read()

        content_list = file_content.split("\n")
        text_file.close()

        self.sents = content_list

    def lines2sentences(self, lines):   # 문장의 끝맺음이 제대로 되지 않을때 문장 분리
        sentences = sent_tokenize(lines)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx - 1] += (' ' + sentences[idx])
                sentences[idx] = ''

        self.sents = sentences

    def komoran_tokenizer(self, sent):
        words = self.komoran.pos(sent, join=True)
        words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
        return words

    def set_summary_line_cnt(self, cnt):
        self.cnt = cnt

    def run(self):
        summarizer = KeysentenceSummarizer(
            tokenize=self.komoran_tokenizer,
            min_sim=0.3,
            verbose=False
        )
        keysents = summarizer.summarize(self.sents, topk=self.cnt)
        return keysents


# 함수 시연
## word_cloud / 단어 빈도 분석

In [17]:
lines = "파이썬은 초보자부터 전문가까지 사용자층을 보유하고 있다. 동적 타이핑(dynamic typing) 범용 프로그래밍 언어로, 펄 및 루비와 자주 비교된다. 다양한 플랫폼에서 쓸 수 있고, 라이브러리(모듈)가 풍부하여, 대학을 비롯한 여러 교육 기관, 연구 기관 및 산업계에서 이용이 증가하고 있다. 또 파이썬은 순수한 프로그램 언어로서의 기능 외에도 다른 언어로 쓰인 모듈들을 연결하는 접착제 언어로써 자주 이용된다. 실제 파이썬은 많은 상용 응용 프로그램에서 스크립트 언어로 채용되고 있다. 도움말 문서도 정리가 잘 되어 있으며, 유니코드 문자열을 지원해서 다양한 언어의 문자 처리에도 능하다."

word_cloud_result = word_cloud(lines)
print(word_cloud_result)

{'fields': [{'name': '유형', 'type': 'TEXT'}, {'name': '개수', 'type': 'INTEGER'}], 'results': [['언어', 6], ['파이썬', 3], ['이용', 2], ['기관', 2], ['모듈', 2], ['및', 2], ['프로그램', 2], ['문서', 1], ['말', 1], ['문자', 1], ['기능', 1], ['외', 1], ['지원', 1], ['연결', 1], ['접착제', 1], ['문자열', 1], ['실제', 1], ['상용', 1], ['응용', 1], ['스크립트', 1], ['순수', 1], ['유니코드', 1], ['도움', 1], ['정리', 1], ['채용', 1], ['산업', 1], ['증가', 1], ['펄', 1], ['전문가', 1], ['사용', 1], ['보유', 1], ['동적', 1], ['타이핑', 1], ['범용', 1], ['프로그래밍', 1], ['루비', 1], ['초보자', 1], ['비교', 1], ['플랫폼', 1], ['수', 1], ['라이브러리', 1], ['대학', 1], ['교육', 1], ['연구', 1], ['처리', 1]]}


## word_relative / 연관어 분석

In [18]:
lines = "파이썬은 초보자부터 전문가까지 사용자층을 보유하고 있다. 동적 타이핑(dynamic typing) 범용 프로그래밍 언어로, 펄 및 루비와 자주 비교된다. 다양한 플랫폼에서 쓸 수 있고, 라이브러리(모듈)가 풍부하여, 대학을 비롯한 여러 교육 기관, 연구 기관 및 산업계에서 이용이 증가하고 있다. 또 파이썬은 순수한 프로그램 언어로서의 기능 외에도 다른 언어로 쓰인 모듈들을 연결하는 접착제 언어로써 자주 이용된다. 실제 파이썬은 많은 상용 응용 프로그램에서 스크립트 언어로 채용되고 있다. 도움말 문서도 정리가 잘 되어 있으며, 유니코드 문자열을 지원해서 다양한 언어의 문자 처리에도 능하다."


word_relative_result = word_relative(lines)
print(word_relative_result)

{'fields': [{'name': 'id', 'type': 'TEXT'}, {'name': 'link', 'type': 'TEXT'}, {'name': 'radius', 'type': 'INTEGER'}, {'name': 'color', 'type': 'TEXT'}], 'results': [['파이썬은', '파이썬은', 16, '#ff0000'], ['언어로', '언어로', 15, '#ff1919'], ['자주', '자주', 14, '#ff3333'], ['프로그램', '프로그램', 13, '#ff4c4c'], ['접착제', '접착제', 13, '#ff4c4c'], ['쓰인', '쓰인', 13, '#ff4c4c'], ['언어로써', '언어로써', 13, '#ff4c4c'], ['다른', '다른', 12, '#ff6666'], ['언어로서의', '언어로서의', 12, '#ff6666'], ['연결하는', '연결하는', 12, '#ff6666'], ['순수한', '순수한', 12, '#ff6666'], ['이용된다.', '이용된다.', 12, '#ff6666'], ['모듈들을', '모듈들을', 12, '#ff6666'], ['있다.', '있다.', 11, '#ff7f7f'], ['또', '또', 6, '#ff9999'], ['외에도', '외에도', 4, '#ffb2b2'], ['많은', '많은', 3, '#ffcccc'], ['상용', '상용', 2, '#ffe5e5'], ['채용되고', '채용되고', 2, '#ffe5e5'], ['기능', '기능', 2, '#ffe5e5'], ['전문가까지', '전문가까지', 1, '#ffffff'], ['응용', '응용', 1, '#ffffff'], ['스크립트', '스크립트', 1, '#ffffff'], ['프로그램에서', '프로그램에서', 1, '#ffffff'], ['실제', '실제', 1, '#ffffff'], ['언어로', '파이썬은', 2, '#bfbfbf'], ['있다.', '파이썬은', 2, '#bfbfbf'

## key_sentence / 핵심 문장 추출

In [19]:
lines = "파이썬은 초보자부터 전문가까지 사용자층을 보유하고 있다. 동적 타이핑(dynamic typing) 범용 프로그래밍 언어로, 펄 및 루비와 자주 비교된다. 다양한 플랫폼에서 쓸 수 있고, 라이브러리(모듈)가 풍부하여, 대학을 비롯한 여러 교육 기관, 연구 기관 및 산업계에서 이용이 증가하고 있다. 또 파이썬은 순수한 프로그램 언어로서의 기능 외에도 다른 언어로 쓰인 모듈들을 연결하는 접착제 언어로써 자주 이용된다. 실제 파이썬은 많은 상용 응용 프로그램에서 스크립트 언어로 채용되고 있다. 도움말 문서도 정리가 잘 되어 있으며, 유니코드 문자열을 지원해서 다양한 언어의 문자 처리에도 능하다."


key_sentence_result = key_sentence(lines)
print(key_sentence_result)

{'fields': [{'name': '핵심 문장 추출', 'type': 'TEXT'}], 'results': [['또 파이썬은 순수한 프로그램 언어로서의 기능 외에도 다른 언어로 쓰인 모듈들을 연결하는 접착제 언어로써 자주 이용된다.'], ['다양한 플랫폼에서 쓸 수 있고, 라이브러리(모듈)가 풍부하여, 대학을 비롯한 여러 교육 기관, 연구 기관 및 산업계에서 이용이 증가하고 있다.'], ['도움말 문서도 정리가 잘 되어 있으며, 유니코드 문자열을 지원해서 다양한 언어의 문자 처리에도 능하다.'], ['실제 파이썬은 많은 상용 응용 프로그램에서 스크립트 언어로 채용되고 있다.'], ['동적 타이핑(dynamic typing) 범용 프로그래밍 언어로, 펄 및 루비와 자주 비교된다.']]}
